In [1]:
# lstm model
import numpy as np
import pandas as pd
from numpy import array
from numpy import mean
from numpy import std
from numpy import dstack
import scipy.io as sio
from pandas import read_csv
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot
import os

Using TensorFlow backend.


In [2]:
# load all movements

skelList = list()
exerciseList = list()

for i in range(1,17):
    
    path, dirs, files = next(os.walk("data/person%i" %i))
    file_count = len(files)
    
    for j in range(1,file_count):
        
        mat = sio.loadmat('data/person%i/myfile%s.mat' %(i,str(j)))
        skelSeq = mat['skelSeq']
        skelSeq = np.around(skelSeq, decimals = 4)
        exerciseNum = mat['exerciseNum']
        skelList.append(skelSeq)
        exerciseList.append(exerciseNum[0].item(0))

C:\Users\Mahya\Anaconda3\lib\site-packages\scipy\io\matlab\mio.py:208: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [3]:
exerciseList = array(exerciseList)
print(exerciseList.shape)

(843,)


In [4]:
skelList[0].shape
print(type(skelList))

<class 'list'>


In [5]:
for i in range(1,len(skelList)):
    skelList[i] = np.reshape(skelList[i], (75, skelList[i].shape[1]))

print(type(skelList))
skelList = array(skelList)
print(skelList.shape)
print(type(skelList))

<class 'list'>
(843,)
<class 'numpy.ndarray'>


In [6]:
# load the dataset, returns train and test X and y elements# 
X_train, X_test, y_train, y_test = train_test_split(skelList, exerciseList, test_size=0.33, random_state=42)

In [31]:
n_timesteps =3
num_features =2
nb_samples = 224
x_train_reshaped = np.zeros((nb_samples, look_back, num_features))
y_train_reshaped = np.zeros((nb_samples))
print("nb_samples",nb_samples)
print(X_train.shape)

# x_train_reshaped = np.reshape(X_train,(188,3,1))
# y_train_reshaped = np.reshape(y_train,(188))

for i in range(nb_samples):
    y_position = i + look_back
#     x_train_reshaped[i] = X_train[i:y_position]
    y_train_reshaped[i] = y_train[y_position]
 
  
print( y_train_reshaped.shape)


nb_samples 224
(564,)
(224,)


In [42]:
x_test_reshaped = np.zeros((nb_samples, look_back, num_features))
y_test_reshaped = np.zeros((nb_samples))
print("nb_samples",nb_samples)
print(X_test.shape)

x_train_reshaped = np.reshape(X_train,(188,3,1))
y_train_reshaped = np.reshape(y_train,(188))

for i in range(nb_samples):
    y_position = i + look_back
#     x_train_reshaped[i] = X_train[i:y_position]
    y_test_reshaped[i] = y_test[y_position]
 
  
# print( y_test_reshaped.shape)

nb_samples 224
(279,)
(224,)


In [ ]:
# split into samples (e.g. 564/12 = 47)
# samples = list()
# length = 12

# step over the 5,000 in jumps of 200
# for i in range(0,len(X_train),length):
#     # grab from i to i + 200
#     sample = X_train[i:i+length]
#     samples.append(sample)
# print(len(samples))

In [ ]:
# samples =array(samples)
# print(samples.shape)

In [ ]:
# samples = samples.reshape(len(samples),length,1)
# print(samples.shape)

In [ ]:
ysamples = list()
length = 2

# step over the 5,000 in jumps of 200
for i in range(0,len(y_train),length):
    # grab from i to i + 200
    ysample = y_train[i:i+length]
    ysamples.append(ysample)
print(len(ysamples))

ysamples =array(ysamples)
print(ysamples.shape)

ysamples = ysamples.reshape(len(ysamples),length,1)
print(ysamples.shape)

In [44]:
# fit and evaluate a model

# 1 secons has 30 frames. we consider every movement takes 2s and15 more frames for overlapping
#3*25=75 feature
# n_timesteps, n_features= 75,75

model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,num_features)))
model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))
model.add(Dense(14, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network

model.fit(x_train_reshaped, y_train_reshaped, epochs=200, batch_size= len(skelList), verbose= 0)
# # evaluate model
_, accuracy = model.evaluate(x_test_reshaped, y_test_reshaped, batch_size=64, verbose=0)

In [45]:
print("accuracy",accuracy)

accuracy 0.1294642835855484
